In [ ]:
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.timeseries.output_writer import OutputWriter


# Install pandapower in a new environment by pip install pandapower==2.13.1
net = pp.from_pickle('C:/Users/bjorn/OneDrive/Documents/TU Delft/BEP/TUD Library/bap/aggregated_grid_2018_with_generators_loads_costs_controllers.p')

out_path = "C:/Users/bjorn/OneDrive/Documents/TU Delft/BEP/TUD Library/bap/OPF"
ow = OutputWriter(net, output_path=out_path, output_file_type=".csv", csv_separator=",")
# ow.remove_log_variable("res_gen", "vm_pu")

In [ ]:
# TIMESTEP-WISE SIMULATION TO GET SIMULATED VALUES


# Single time values
def get_simulated_values(t):
    for controller in net.controller.object:
        controller.time_step(t, net)

    pp.runpp(net)

In [ ]:
# GET FROM SIMULATION

def get_from_simulation(res_csv, component_numbers):
    # Join simulation timeseries dataframes in a dataframe
    bus_timesteps = pd.concat([res_csv[f"{g}"] for g in component_numbers], axis=1)

    return bus_timesteps

In [ ]:
# GET FROM CONTROLLER

def get_from_controller(start_controller_number, component_numbers):
    # Join controller timeseries dataframes at the same bus in a dataframe
    bus_timesteps = pd.concat([net.controller.object[start_controller_number + g].data_source.to_dict()['df'].iloc[:,0] for g in component_numbers], axis=1)

    return bus_timesteps

In [ ]:
# GET AGGREGATED DATA

total_bus_numbers = net.bus.index

def get_aggregated_x(res_x, bus_gens, get_from):
    # res_x = csv for get_from_csv or start controller number for get_from_controller
    gen_busses_x = [] # List of timestep-wise gen x dataframes for each bus

    for bus in bus_gens:
        # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
        bus_timesteps = get_from(res_x, bus)

        # Add timestep-wise values together
        bus_timesteps_total = bus_timesteps.sum(axis=1)
        
        gen_busses_x.append(bus_timesteps_total)

    gen_x = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

    gen_x.columns = bus_gens.index # Column names to bus numbers

    gen_x = gen_x.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
    
    return gen_x



# gen_busses_q_mvar = [] 
# gen_busses_vm_pu = []
# gen_busses_va_degree = []



# for bus in bus_gens:
#     # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
#     bus_timesteps = pd.concat([res_q_mvar[f"{g}"] for g in bus], axis=1)
#     bus_timesteps_vm = pd.concat([res_vm_pu[f"{vm}"] for vm in bus], axis=1)
#     bus_timesteps_va = pd.concat([res_va_degree[f"{va}"] for va in bus], axis=1)

#     # Add timestep-wise values together
#     bus_timesteps_total = bus_timesteps.sum(axis=1)
#     bus_timesteps_total_vm = bus_timesteps_vm.sum(axis=1)
#     bus_timesteps_total_va = bus_timesteps_va.sum(axis=1)

#     gen_busses_x.append(bus_timesteps_total)
#     gen_busses_vm_pu.append(bus_timesteps_total_vm)    
#     gen_busses_va_degree.append(bus_timesteps_total_va)

# gen_q_mvar = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_vm_pu = pd.concat(gen_busses_vm_pu, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_va_degree = pd.concat(gen_busses_va_degree, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

# gen_q_mvar.columns = bus_gens.index # Column names to bus numbers
# gen_vm_pu.columns = bus_gens.index # Column names to bus numbers
# gen_va_degree.columns = bus_gens.index # Column names to bus numbers

# gen_q_mvar = gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_vm_pu = gen_vm_pu.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_va_degree = gen_va_degree.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0

# gen_vm_pu.head(100)    


In [ ]:
# CREATE LOAD Q_MVAR

pf = 0.95 #Using a power factor 0.95
p_to_q = np.sin(np.arccos(pf))/pf # P to Q constant

loads_p_mw = get_from_controller(0, net.load.index) # All controller p_mw data

loads_q_mvar = loads_p_mw.mul(p_to_q)

loads_q_mvar.head(50)


In [ ]:
# CREATE LOAD Q_MVAR CONTROLLER

for l in range(len(net.load.index)):
    controller_load_q_mvar = loads_p_mw.iloc[:, l].multiply(p_to_q)
    dataset = controller_load_q_mvar.to_frame()
    prof_name = dataset.columns[0]
    dataset = pp.timeseries.data_sources.frame_data.DFData(dataset)
    
    pp.control.controller.const_control.ConstControl(net, "load", "q_mvar", element_index=l, 
                                                 data_source=dataset, 
                                                 recycle={'trafo': False, 'gen': False, 'bus_pq': True}, 
                                                 profile_name=prof_name)

net.controller

In [ ]:
# GET SIMULATION RESULTS

ow.log_variable('res_bus', 'vm_pu') # add logging for bus voltage magnitudes
ow.log_variable('res_bus', 'va_degree') # add logging for bus voltage degree
ow.log_variable('res_bus', 'p_mw') # add logging for bus voltage power
ow.log_variable('res_bus', 'q_mvar') # add logging for bus voltage reactive power

ow.log_variable('res_line', 'loading_percent') # add logging for line loadings in percent

ow.log_variable('res_gen', 'p_mw')
ow.log_variable('res_gen', "q_mvar")
ow.log_variable('res_gen', "va_degree")
ow.log_variable('res_gen', "vm_pu")

ow.log_variable('res_sgen', 'p_mw')
ow.log_variable('res_sgen', 'q_mvar')

ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_load', 'q_mvar')

# def get_simulated_timeseries(t_init, t_total):
#     pp.timeseries.run_time_series.run_timeseries(net, time_steps=range(t_init,t_init + t_total))

# get_simulated_timeseries(0, 24)

pp.timeseries.run_timeseries(net, run=pp.rundcopp)

In [ ]:
# SWAPPING SLACK BUS (35)

def swap_slack(df, slack_number): # Switch slack bus position
    df.iloc[:, 0], df.iloc[:, slack_number] = df.iloc[:, slack_number], df.iloc[:, 0]

slack_bus = 35

In [ ]:
# GET GENERATION P_MW

gen_active = net.gen.loc[net.gen["in_service"] == True] # Drop unused generators
bus_gens = gen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group generators by bus

res_gen_p_mw = pd.read_csv(out_path + "/res_gen/p_mw.csv") # Read from simulation result

gen_p_mw = get_aggregated_x(res_gen_p_mw, bus_gens, get_from_simulation) # Get bus-aggregated generation

swap_slack(gen_p_mw, slack_bus)

# gen_p_mw.head(50)

In [ ]:
# GET BUS VM_PU
# 1 if PV or slack node, else unknown
slack_active = net.ext_grid.loc[net.ext_grid["in_service"] == True]

vm_pu = pd.Series([1 if (i in bus_gens.index or i in slack_active.bus.to_list()) 
                   else None for i in total_bus_numbers])

vm_pu[0], vm_pu[slack_bus] = vm_pu[slack_bus], vm_pu[0]

# vm_pu

In [ ]:
# GET LOAD P_MW

bus_loads = net.load.groupby("bus").apply(lambda x: x.index.tolist()) # Group loads by bus

load_p_mw = get_aggregated_x(0, bus_loads, get_from_controller)

# Print load P_MW for forecasting:
# load_p_mw.to_csv(out_path + "/total_load.csv")

swap_slack(load_p_mw, slack_bus)

load_p_mw.head(50)

In [ ]:
# GET LOAD Q_MVAR

load_q_mvar = load_p_mw.mul(p_to_q)

load_q_mvar.head(50)

In [ ]:
# GET SGEN P_MW

sgen_active = net.sgen.loc[net.sgen["in_service"] == True] # Drop unused sgens
bus_sgens = sgen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group sgens by bus

res_sgen_p_mw = pd.read_csv(out_path + "/res_sgen/p_mw.csv") # Read from simulation result

sgen_p_mw = get_aggregated_x(res_sgen_p_mw, bus_sgens, get_from_simulation) # Get bus aggregated sgen

swap_slack(sgen_p_mw, slack_bus)

sgen_p_mw.head(50)


In [ ]:
# SGEN Q_MVAR = 0

sgen_q_mvar = np.zeros(len(total_bus_numbers))

In [ ]:
# GET SLACK WEIGHT

slack_weight = [1 if i in slack_active.bus.to_list() else 0 for i in total_bus_numbers]
slack_weight[0], slack_weight[slack_bus] = slack_weight[slack_bus], slack_weight[0]

slack = slack_weight # Slack yes or no

print(slack_weight)

In [ ]:
# GET EMPTY BUSES

active_bus = set(net.load.bus.to_list())
active_bus.update(gen_active.bus.to_list())
active_bus.update(sgen_active.bus.to_list())
active_bus.update(slack_active.bus.to_list())

empty_bus = [bus for bus in total_bus_numbers if bus not in active_bus]

for i range(len(empty_bus)):
    if empty_bus[i] == 0:
        empty_bus[i] = slack_bus
    elif empty_bus[i] == slack_bus:
        empty_bus[i] = 0

print(empty_bus)

In [ ]:
# GET BUS TYPE

bus_type = np.where(vm_pu == 1, 2, 3) # 2 if PV (or slack), 3 if PQ (or empty/not in service)
bus_type[0] = 1

print(bus_type)

In [ ]:
# GET BUS ANGLE

va_degree = np.where(np.array(slack) == 1, 0, None)

print(va_degree)

In [ ]:
# JOIN LOAD AND SGEN

load_sgen_p_mw = load_p_mw.subtract(sgen_p_mw)
load_sgen_q_mvar = load_q_mvar.subtract(sgen_q_mvar)

load_sgen_p_mw.head(5)

In [ ]:
# GET MAX AND MIN GEN MVAR

min_gen_q_mvar = gen_active.groupby("bus")["min_q_mvar"].sum()
min_gen_q_mvar = min_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none

max_gen_q_mvar = gen_active.groupby("bus")["max_q_mvar"].sum()
max_gen_q_mvar = max_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none

min_gen_q_mvar

In [ ]:
# GET USER SOLAR AND WIND BUSSES

extra_bus = pd.Series([2, 1, None, None, None, 0, 0, 1000000, -1000000, 0, 0])
extra_buses = pd.DataFrame([extra_bus, extra_bus])
extra_buses.columns = ["Type", "PU Volt", "Angle (Deg)", "Gen MW", "Gen Mvar", "Load MW", "Load Mvar", 
                       "Gen Mvar(max)", "Gen Mvar(min)", "Slack yes,no", "Slack Weight"]
extra_buses

In [ ]:
def to_pu(power, sbase=100.0):
    return np.divide(power, sbase)

In [ ]:
# GETTING ALL DATA FOR ONE TIMESTEP

nones = [None for _ in total_bus_numbers]

def get_timestep(t):
    timestep = pd.DataFrame(data={"Type": bus_type, "PU Volt": vm_pu.values, "Angle (Deg)":va_degree, "Gen MW": to_pu(gen_p_mw.iloc[t].values), 
                                  "Gen Mvar": nones, "Load MW": to_pu(load_sgen_p_mw.iloc[t].values), 
                                  "Load Mvar": to_pu(load_sgen_q_mvar.iloc[t].values), "Gen Mvar(max)": to_pu(max_gen_q_mvar), 
                                  "Gen Mvar(min)": to_pu(min_gen_q_mvar), "Slack yes,no": slack, "Slack Weight": slack_weight})
    
    timestep = pd.concat([timestep, extra_buses], axis=0, ignore_index=True) # Add solar and wind bus
    
    return timestep

get_timestep(0).head(50)


In [ ]:
# CREATING THE CSV WITH ALL TIMESTEPS

def create_csv(t_init, t_duration, index=True):
    result = pd.concat([get_timestep(t) for t in range(t_init, t_init + t_duration)], axis=0)
    
    result.to_csv(out_path + "/busInputData.csv", index=index)

In [ ]:
create_csv(0, 72, index=False)

In [ ]:
# TESTING OPF

res_load_p_mw = pd.read_csv(out_path + "/res_load/p_mw.csv") # Read from simulation result
res_load_q_mvar = res_load_p_mw.mul(p_to_q)

def run_simulation(t_init=0, t_final=8760, function=pp.runpp):

    # init_load = net.load[["p_mw", "q_mvar"]]

    net.gen["vm_pu"] = np.ones(len(net.gen)) # Set to 1
    net.sgen["q_mvar"] = np.zeros(len(net.sgen)) # Set to 0
    net.ext_grid["vm_pu"] = np.ones(len(net.ext_grid))
    net.ext_grid["va_degree"] = np.zeros(len(net.ext_grid))

    res_lp = []
    res_lq = []

    res_gp = []
    res_gq = []
    res_gvm = []
    res_gva = []

    res_sgp = []
    res_sgq = []

    res_bp = []
    res_bq = []
    res_bvm = []
    res_bva = []

    res_l = []

    for t in range(t_init, t_final):
        net.load["p_mw"] = res_load_p_mw.iloc[t, 1:].values
        net.load["q_mvar"] = res_load_q_mvar.iloc[t, 1:].values

        net.gen["p_mw"] = res_gen_p_mw.iloc[t, 1:].values

        net.sgen["p_mw"] = res_sgen_p_mw.iloc[t, 1:].values

        function(net)

        res_lp.append(net.res_load["p_mw"].copy().to_frame().T) # load_p_mw
        res_lq.append(net.res_load["q_mvar"].copy().to_frame().T) # load_q_mvar

        res_gp.append(net.res_gen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_gq.append(net.res_gen["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_gvm.append(net.res_gen["vm_pu"].copy().to_frame().T) # gen_vm_pu
        res_gva.append(net.res_gen["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_sgp.append(net.res_sgen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_sgq.append(net.res_sgen["q_mvar"].copy().to_frame().T) # gen_p_mw

        res_bp.append(net.res_bus["p_mw"].copy().to_frame().T) # gen_q_mvar
        res_bq.append(net.res_bus["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_bvm.append(net.res_bus["vm_pu"].copy().to_frame().T) # gen_q_mvar
        res_bva.append(net.res_bus["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_l.append(net.res_line["loading_percent"].copy().to_frame().T) # gen_q_mvar

    
    res_lp = pd.concat(res_lp, axis=0)
    res_lq = pd.concat(res_lq, axis=0)

    res_gp = pd.concat(res_gp, axis=0)
    res_gq = pd.concat(res_gq, axis=0)
    res_gvm = pd.concat(res_gvm, axis=0)
    res_gva = pd.concat(res_gva, axis=0)

    res_sgp = pd.concat(res_sgp, axis=0)
    res_sgq = pd.concat(res_sgq, axis=0)

    res_bp = pd.concat(res_bp, axis=0)
    res_bq = pd.concat(res_bq, axis=0)
    res_bvm = pd.concat(res_bvm, axis=0)
    res_bva = pd.concat(res_bva, axis=0)

    res_l = pd.concat(res_l, axis=0)

    return {"load": [res_lp, res_lq], "gen": [res_gp, res_gq, res_gvm, res_gva], "sgen": [res_sgp, res_sgq], 
            "bus": [res_bp, res_bq, res_bvm, res_bva], "line": [res_l]}


results = run_simulation(t_final=5, function=pp.runpp)


In [ ]:
results["bus"][1].head()